# 제주도 핫플레이스 - 인스타그램 크롤링

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd 
from urllib.parse import quote

In [2]:
chromedriver = '/Users/TECH-8/Downloads/chromedriver.exe'
# chromedriver = 'C:\\Users\\TECH-8\\Downloads\\chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

### 1. Instagram 에 접속하여 로그인 

In [3]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [4]:
with open('password.txt') as f:
    password = f.read(8)

In [5]:
email = 'ckiekim'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [6]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [7]:
# 로그인 정보 저장 - 나중에 하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [8]:
# 알림 설정 - 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 2. 제주도맛집 으로 검색

In [29]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [30]:
driver.get(url)
time.sleep(5)

### 3. 첫번째 게시글 열기

In [31]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

### 4. 게시글 정보 가져오기

In [32]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [33]:
# 1) 본문 내용 가져오기
import unicodedata
try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'.< 제주해물라면 맛집모음! > 일단저장@ 해물 아낌없이 주는 해물라면 맛집!바다뷰보며 먹는 해물라면은 말이 필요없음!저장해뒀다가 다음 여행시 참고하기!함께 하고픈 친구소환@..#제주맛집 #제주도맛집 #제주공항근처맛집 #제주시맛집 #애월맛집 #중문맛집 #제주공항맛집 #서귀포맛집 #산방산맛집 #신제주맛집 #제주시청맛집 #제주애월맛집 #월정리맛집 #제주여행 #제주핫플'

In [34]:
# 2) 본문 내용에서 해쉬태그 가져오기, '#[^\s#,\\]+'
import re
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#제주맛집',
 '#제주도맛집',
 '#제주공항근처맛집',
 '#제주시맛집',
 '#애월맛집',
 '#중문맛집',
 '#제주공항맛집',
 '#서귀포맛집',
 '#산방산맛집',
 '#신제주맛집',
 '#제주시청맛집',
 '#제주애월맛집',
 '#월정리맛집',
 '#제주여행',
 '#제주핫플']

In [35]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-08-18'

In [36]:
# 4) 좋아요 수 가져오기: 좋아요 1,296개
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

0

In [37]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

''

In [38]:
row = [content, date, like, place, tags]
row

['.< 제주해물라면 맛집모음! > 일단저장@ 해물 아낌없이 주는 해물라면 맛집!바다뷰보며 먹는 해물라면은 말이 필요없음!저장해뒀다가 다음 여행시 참고하기!함께 하고픈 친구소환@..#제주맛집 #제주도맛집 #제주공항근처맛집 #제주시맛집 #애월맛집 #중문맛집 #제주공항맛집 #서귀포맛집 #산방산맛집 #신제주맛집 #제주시청맛집 #제주애월맛집 #월정리맛집 #제주여행 #제주핫플',
 '2021-08-18',
 0,
 '',
 ['#제주맛집',
  '#제주도맛집',
  '#제주공항근처맛집',
  '#제주시맛집',
  '#애월맛집',
  '#중문맛집',
  '#제주공항맛집',
  '#서귀포맛집',
  '#산방산맛집',
  '#신제주맛집',
  '#제주시청맛집',
  '#제주애월맛집',
  '#월정리맛집',
  '#제주여행',
  '#제주핫플']]

### 5. 다음 게시글로 이동

In [39]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(2)

In [59]:
# 6을 하기 위해 popup 화면을 없앰(즉, X 를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

### 6. 여러개의 게시글 가져오기

In [48]:
from tqdm.notebook import tqdm

In [54]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, tags, date, like, place]

In [58]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [56]:
results = []
for _ in tqdm(range(500)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)

  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [51]:
df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
df.head()

,content,date,like,place,tags
0,<광고>섭지코지에서 한끼 맛있게 드시고 싶다면...특히 점심메뉴로 강력추천드리고요 ...,2021-08-18,0,,"[#섭지코지한끼🍱오픈, #섭지코지맛집, #섭지코지, #섭지코지카페, #스누피가든, ..."
1,.#연동 #모니카옛날통닭간판부터 맛집포스 작렬하는 찐맛집이젠 인생치킨이라고 말하기도...,2021-08-19,"1,092",,"[#연동, #모니카옛날통닭간판부터, #제주맛집, #제주도맛집, #제주공항근처맛집, ..."
2,.< 제주해물라면 맛집모음! > 일단저장@ 해물 아낌없이 주는 해물라면 맛집!바다뷰...,2021-08-18,"1,008",,"[#제주맛집, #제주도맛집, #제주공항근처맛집, #제주시맛집, #애월맛집, #중문맛..."
3,.한림 #문도지오름방목된 조랑말과 교감까지 할 수이따규~일몰 시간에 맞춰서 가면 멋...,2021-08-17,"1,303",,"[#문도지오름방목된, #제주맛집, #제주도맛집, #제주공항근처맛집, #제주시맛집, ..."
4,그 유명한 #연돈 에 다녀왔습니다. 그런데ㅎ;;;⠀⠀⠀⠀⠀⠀⠀어마무시한 웨이팅으로 ...,2021-08-18,"4,535",연돈,"[#연돈, #제주도연돈, #테이블링, #등심까스, #수제볼카츠, #아토짱_제주, #..."


In [52]:
df.tail()

,content,date,like,place,tags
195,제주 역대급 돈까스맛집🍴뭐든 푸짐푸짐한거ㅇㅈ?쩝쩝박사님들ㄲㄲ⠀@@내최애는 고구마치즈...,2021-08-19,0,,"[#애월맛집, #애월맛집추천, #애월밥집, #애월돈까스, #애월데이트코스, #애월핫..."
196,흑돼지는 정직한돈‼매일먹어도 먹고싶은내최애맛집 제발머겅❤ 🔥⠀@@오늘갈랭?🤭⠀⠀#애...,2021-08-19,0,,"[#애월맛집, #협재맛집, #한림맛집, #애월흑돼지, #협재흑돼지, #한림흑돼지, ..."
197,신혼여행 1일차😍생각해보니 결혼식 관련해서 아무것도 안올렸넹..누가보면 결혼식 못한...,2021-08-19,23,,"[#제주도, #신혼여행, #3개월지났지만, #올린당, #지난신혼여행, #낭만없음, ..."
198,육즙대잔치라는 정직한돈👏🏻셀럽들이 인정하는곳이래안가면 손해👏🏻👏🏻👏🏻⠀@@메모해🥰...,2021-08-19,0,,"[#애월맛집, #협재맛집, #한림맛집, #애월흑돼지, #협재흑돼지, #한림흑돼지, ..."
199,흑돼지는 정직한돈‼매일먹어도 먹고싶은내최애맛집 제발머겅❤ 🔥⠀@@오늘갈랭?🤭⠀⠀#애...,2021-08-19,0,,"[#애월맛집, #협재맛집, #한림맛집, #애월흑돼지, #협재흑돼지, #한림흑돼지, ..."
